In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, RocCurveDisplay, roc_curve,auc, precision_recall_curve, average_precision_score, f1_score, auc 
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from imblearn.metrics import geometric_mean_score
from copy import deepcopy
import json
import seaborn as sns
import warnings
import pickle
import shap
warnings.filterwarnings("ignore")
sns.set_theme()

from tqdm import tqdm

In [4]:
for i in range(1,1+1):
    
    X_train = pd.read_csv(f'C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/Analysis dataset/Data for WS/[24-12-12]/X_train/X_train_{i}.csv')
    X_test = pd.read_csv(f'C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/Analysis dataset/Data for WS/[24-12-12]/X_test/X_test_{i}.csv')    
    y_train = pd.read_csv(f'C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/Analysis dataset/Data for WS/[24-12-12]/y_train/y_train_{i}.csv')
    y_test = pd.read_csv(f'C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/Analysis dataset/Data for WS/[24-12-12]/y_test/y_test_{i}.csv')

    globals()[f'X_train_{i}'] = X_train.drop(X_train.columns[0], axis=1)
    globals()[f'X_test_{i}'] = X_test.drop(X_test.columns[0], axis=1)
    globals()[f'y_train_{i}'] = y_train.drop(y_train.columns[0], axis=1)
    globals()[f'y_test_{i}'] = y_test.drop(y_test.columns[0], axis=1)

- we performed this analysis when the data are high-risk group.

In [5]:
for i in range(1,1+1):
    
    exec(f'X_train = X_train_{i}')
    exec(f'X_test = X_test_{i}')
    
    exec(f'y_train = y_train_{i}')
    exec(f'y_test = y_test_{i}')
    
    X_train = X_train[X_train['FAMILY_HISTORY']==1][['AGE','MP_OXC','FHXNG','BRCACC_1','BRCACC_2','BMIG']]
    X_test = X_test[X_test['FAMILY_HISTORY']==1][['AGE','MP_OXC','FHXNG','BRCACC_1','BRCACC_2','BMIG']]

    y_train = y_train.iloc[X_train.index]
    y_test = y_test.iloc[X_test.index]
    

    X_train.reset_index(drop=True, inplace=True)
    X_test.reset_index(drop=True, inplace=True)

    globals()[f'X_train_{i}'] = X_train
    globals()[f'X_test_{i}'] = X_test
    
    y_train.reset_index(drop=True)
    y_test.reset_index(drop=True)
    
    globals()[f'y_train_{i}'] = y_train
    globals()[f'y_test_{i}'] = y_test

In [6]:
i = 1

model = pickle.load(open(f"C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/Analysis dataset/Data for WS/[24-12-12]/subgroup analysis/hr/model pkl save/Catboost/fit_catboost_{i}.pkl", 'rb')) 

In [7]:
i = 1

exec(f'X_train = X_train_{i}')
exec(f'X_test = X_test_{i}')

exec(f'y_train = y_train_{i}')
exec(f'y_test = y_test_{i}')

age_mean = np.mean(X_train['AGE'])
age_std = np.std(X_train['AGE'])

X_train['AGE'] = (X_train['AGE'] - age_mean) / age_std
X_test['AGE'] = (X_test['AGE'] - age_mean) / age_std

In [ ]:
round(age_mean,4), round(age_std,4)

### when high risk

In [9]:
my_dict = {'AGE':[0],
           
           'MP_OXC':[0],
           'FHXNG':[1],
           'BRCACC_1':[1],
           'BRCACC_2':[0],
           'BMIG':[1]
          }
my_dict = pd.DataFrame(my_dict)

In [ ]:
my_dict

In [11]:
high_risk = []

for i in range(20,69+1):

    my_dict['AGE'] = i    
    my_dict['AGE'] = (my_dict['AGE'] - age_mean) / age_std
    
    y_pred_probs = model.predict_proba(my_dict)[:,1][0]
    
    high_risk.append(y_pred_probs)

### when non-high risk

In [12]:
my_dict = {'AGE':[0],
           
           'MP_OXC':[0],
           'FHXNG':[0],
           'BRCACC_1':[0],
           'BRCACC_2':[0],
           'BMIG':[0]
          }
my_dict = pd.DataFrame(my_dict)

In [ ]:
my_dict

In [13]:
non_highrisk = []

for i in range(20,69+1):
        
    my_dict['AGE'] = i    
    my_dict['AGE'] = (my_dict['AGE'] - age_mean) / age_std
    
    y_pred_probs = model.predict_proba(my_dict)[:,1][0]

    non_highrisk.append(y_pred_probs)

In [14]:
age = [i for i in range(20,69+1)]
my_dict = {'Age': age, 'high_risk': high_risk, 'non high_risk': non_highrisk}
data = pd.DataFrame(my_dict)

In [ ]:
data

In [16]:
# data.to_csv(f'C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/분석/분석/probability from sample cohort/241224_ps_hr.csv', index=False)

### lifetime risk cumulative incidence plot

In [17]:
prevalence = 0.078055021

In [18]:
beta = (7332/2166)*prevalence

In [ ]:
round(beta,4)

In [22]:
data_calibrated = deepcopy(data)

In [23]:
data_calibrated['high_risk'] = (beta*data_calibrated['high_risk'])/(beta*data_calibrated['high_risk']-data_calibrated['high_risk']+1)
data_calibrated['non high_risk'] = (beta*data_calibrated['non high_risk'])/(beta*data_calibrated['non high_risk']-data_calibrated['non high_risk']+1)

In [24]:
woman_mortal_prob = np.array([0.00027, 0.0003, 0.00032, 0.00032, 
    0.00033, 0.00033, 0.00034, 0.00036, 0.00036,
    0.00037, 0.00038, 0.00039, 0.0004, 0.00042,
    0.00045, 0.00047, 0.00051, 0.00056, 0.00061,
    0.00067, 0.00073, 0.00078, 0.00082, 0.00087,
    0.00093, 0.00101, 0.0011, 0.00118, 0.00125,
    0.00131, 0.00135, 0.00139, 0.00145, 0.00155,
    0.00168, 0.00183, 0.00196, 0.00205, 0.00212,
    0.00224, 0.00244, 0.00268, 0.00295, 0.00318, 
    0.00345, 0.00381, 0.00426, 0.00479, 0.00539, 0.00611])
woman_surv_prob = 1- woman_mortal_prob

data_calibrated['woman_surv_prob'] = woman_surv_prob

In [25]:
data_calibrated['IR_high'] = data_calibrated['high_risk']/3.5
data_calibrated['IR_Low'] = data_calibrated['non high_risk']/3.5

data_calibrated['IR_high*surv'] = data_calibrated['IR_high'] * data_calibrated['woman_surv_prob']
data_calibrated['IR_Low*surv'] = data_calibrated['IR_Low'] * data_calibrated['woman_surv_prob']

In [26]:
data_calibrated['lifetime_hr'] = np.zeros(len(data_calibrated))
data_calibrated['lifetime_nonhr'] = np.zeros(len(data_calibrated))

In [27]:
for i in range(len(data_calibrated)):
    data_calibrated['lifetime_hr'][i] = 1-np.exp(-sum(data_calibrated['IR_high*surv'][0:i+1]))
    data_calibrated['lifetime_nonhr'][i] = 1-np.exp(-sum(data_calibrated['IR_Low*surv'][0:i+1]))

In [ ]:
plt.figure(0)

# 예쁘게 그리기!
plt.plot(data_calibrated['Age'], data_calibrated[['lifetime_hr']], label = 'high risk group', color='#E64B35FF')
plt.plot(data_calibrated['Age'], data_calibrated[['lifetime_nonhr']], label = 'non-high risk group', color='#3C5488FF')
plt.ylim(0, 1.0)
plt.xlabel('Age')
plt.ylabel('Cumulative incidence (%)')
plt.title('Cumulative incidence plot using lifetime risk')
plt.legend(loc='best')


# plt.savefig('C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/Figure/lifetime risk ci plot/[24-12-05]/ci plot.tiff', format='tiff', dpi=300)  
# plt.savefig('C:/Users/chaehyun/Dropbox/Work/PIPET/과제/산부인과/난소암 연구/Figure/lifetime risk ci plot/[24-12-05]/ci plot.pdf', format='pdf', dpi=300)  
# plt.close()  